In [1]:
import requests 
from bs4 import BeautifulSoup as soup
import re
import pandas as pd

In [2]:
def linksRetrieve(artistName, listName, url, targetString, nameList):
    '''Creates a link list and saves it in a CSV file'''
    artistName=requests.get(url)
    pageHTML=soup(artistName.text, 'html.parser')
    listName=[]
    for link in pageHTML.find_all('a', attrs={'href':re.compile(targetString)}): 
        listName.append(link.get('href')) 
    listName=pd.DataFrame(listName)
    listName.to_csv(nameList+'.csv', header=['links'])

In [3]:
def lyricRetrieve(linkFile, artistName):
    '''Retrieves lyrics from every link in file and saves them into a nested list for another CSV file'''
    links=pd.read_csv(linkFile)
    songDF=[] 
    for i in range(len(links)):
        url=links['links'].iloc[i]  
        songLink=requests.get(url) 
        songHTML=soup(songLink.text, 'html.parser') 
        lines=''  
        result=songHTML.find_all('p', {'class':"verse"})
        if not result:
            continue    
        for line in songHTML.find_all('p', {'class':"verse"}): 
            line=str(line.text).replace('\'', '%temp%').replace('\n', ' ').replace('%temp%', ' ')+' ' 
            line=line.replace('(', '%temp%').replace(')', ' ').replace('%temp%', ' ')
            lines=lines+line 
        songDF.append(lines) 
    songDF=pd.DataFrame(songDF)
    songDF.to_csv(artistName+'.csv', header=['songs'])

In [4]:
url='https://www.metrolyrics.com/lady-gaga-lyrics.html'
linksRetrieve('LadyGaga', 'gagaSongs', url, '-lyrics-lady-gaga', 'gagaLinks')
lyricRetrieve('gagaLinks.csv', 'gagaLyrics')

In [5]:
url='https://www.metrolyrics.com/frank-sinatra-lyrics.html'
linksRetrieve('FrankSinatra', 'sinatraSongs', url, '-lyrics-frank', 'sinatraLinks')
lyricRetrieve('sinatraLinks.csv', 'sinatraLyrics')